<a href="https://colab.research.google.com/github/arnavt1605/Projects/blob/main/DL/CIFAR10_ResNet18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import torch
import torch.nn as nn
import torch.optim
import torchvision
from torchvision import transforms
from torchvision.models import resnet18
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from torch.utils.data import random_split

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
mean = (0.4914, 0.4822, 0.4465)
std = (0.2023, 0.1994, 0.2010)

print("Mean: ", mean)
print("Standard Deviation: ", std)

Mean:  (0.4914, 0.4822, 0.4465)
Standard Deviation:  (0.2023, 0.1994, 0.201)


In [15]:
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding= 4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(0.2, 0.2 ,0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

test_transform= transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

In [16]:
base_train= CIFAR10(root="data", train=True, download=True, transform= train_transform)
base_val= CIFAR10(root="data", train=True, download=True, transform= test_transform)

torch.manual_seed(0)

train_set, _ = torch.utils.data.random_split(base_train, [45000, 5000])
_, val_set = torch.utils.data.random_split(base_val, [45000, 5000])

test_set = CIFAR10(root="data", train=False, download=True, transform=test_transform)

train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
val_loader = DataLoader(val_set, batch_size=64, shuffle = False)
test_loader = DataLoader(test_set, batch_size=64, shuffle = False)

print(len(train_set), len(test_set), len(val_set))


45000 10000 5000


In [17]:
model = resnet18(weights=None)

model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
model.maxpool = nn.Identity()
model.fc = nn.Linear(model.fc.in_features, 10)

In [18]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", factor= 0.5 ,patience= 3)


In [19]:
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): Identity()
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), p

In [20]:
def train(dataloader, model, loss_function, optimizer):
  model.train()
  total_loss=0

  for batch, (image, label) in enumerate(dataloader):
    image= image.to(device)
    label= label.to(device)

    prediction= model(image)
    loss= loss_function(prediction, label)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    total_loss+= loss

  avg_loss = total_loss/len(dataloader)
  print(f"Training average loss: {avg_loss:.4f}")

In [21]:
def validate(dataloader, model, loss_function):
  model.eval()
  total_loss= 0
  correct= 0
  total= 0
  with torch.no_grad():
    for image, label in dataloader:
      image= image.to(device)
      label= label.to(device)

      pred = model(image)
      loss= loss_function(pred, label)
      total_loss+= loss

      predicted_classes = pred.argmax(dim=1)
      correct+= (predicted_classes == label).sum().item()
      total+= label.size(0)

  avg_loss= total_loss / len(dataloader)
  accuracy= correct/total * 100
  print(f"Validation Loss: {avg_loss:.4f}, Validation Accuracy: {accuracy:.2f}%")

  return accuracy



In [22]:
def test(dataloader, model, loss_function):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for image, label in dataloader:
            image= image.to(device)
            label= label.to(device)

            preds = model(image)
            loss = loss_function(preds, label)
            total_loss += loss.item()

            predicted_classes = preds.argmax(dim=1)
            correct += (predicted_classes == label).sum().item()
            total += label.size(0)

    avg_loss = total_loss / len(dataloader)
    accuracy = correct / total * 100
    print(f"Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.2f}%")

In [23]:
epochs = 30
for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")
    train(train_loader, model, loss_function, optimizer)
    val_accuracy = validate(val_loader, model, loss_function)
    scheduler.step(val_accuracy) #No arguments with StepLR, ReduceLROnPlateau takes arguments


Epoch 1/30
Training average loss: 1.4534
Validation Loss: 1.1027, Validation Accuracy: 60.48%

Epoch 2/30
Training average loss: 1.0244
Validation Loss: 0.8965, Validation Accuracy: 68.62%

Epoch 3/30
Training average loss: 0.8252
Validation Loss: 0.7367, Validation Accuracy: 74.48%

Epoch 4/30
Training average loss: 0.7084
Validation Loss: 0.5944, Validation Accuracy: 79.36%

Epoch 5/30
Training average loss: 0.6260
Validation Loss: 0.5497, Validation Accuracy: 81.64%

Epoch 6/30
Training average loss: 0.5614
Validation Loss: 0.4664, Validation Accuracy: 84.14%

Epoch 7/30
Training average loss: 0.5105
Validation Loss: 0.4360, Validation Accuracy: 85.08%

Epoch 8/30
Training average loss: 0.4714
Validation Loss: 0.3575, Validation Accuracy: 88.18%

Epoch 9/30
Training average loss: 0.4415
Validation Loss: 0.3317, Validation Accuracy: 88.40%

Epoch 10/30
Training average loss: 0.4137
Validation Loss: 0.3143, Validation Accuracy: 89.62%

Epoch 11/30
Training average loss: 0.3809
Valida

In [24]:
test(test_loader, model, loss_function)

Test Loss: 0.3364, Test Accuracy: 90.84%
